# Защита персональных данных клиентов

Необходимо защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

**Цель**

Необходимо защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

**Описание данных**

- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.


## Загрузка данных

In [4]:
# импортируем библиотеки

import pandas as pd
import numpy as np

from sklearn.metrics import r2_score

In [5]:
# загрузим данные

try:
    data = pd.read_csv('/datasets/insurance.csv')
except:
    data = pd.read_csv('insurance.csv')

display(data)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [6]:
# проверим информацию

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
# помсотрим на статистические данные

data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [8]:
# проверим есть ли пропуски

print(f'Количество пропусков по каждому столбцу')
print(data.isna().sum())

Количество пропусков по каждому столбцу
Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64


**Вывод**
- Таблица состоит из 5000 строк и 5 столбцов;
- Все столбцы имеют числовые типы данных;
- Пропусков не обнаружено;
- Судя по результату метода describe выбросов в таблице нет.

В таблице данных компании соблюден почти идеальный баланс между мужчинами и женщинами. Средний возраст застрахованного составляет около 31 года, при этом медиана составляет 30. Границы возраста застрахованных приводят к выводу, что компания не занимается страхованием лиц, чей возраст составляет менее 18 и старше 65 лет. Также интересно взглянуть на уровень заработной платы застрахованных - медиана составляет чуть более 40 000 рублей. В целом, в компании скорее представлен средний экономический класс, чем слишком бедные или слишком богатые граждане. 


## Умножение матриц

**Необходимо ответить на вопрос: если умножить признаки на обратимую матрицу, изменится ли качество линейной регрессии? (Её можно обучить заново.)**

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Если умножить признаки на обратимую матрицу, качество линейной регрессии не изменится.

**Обоснование показано в расчетах ниже:** 

$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$
$$
w = (X^T X)^{-1} X^T y
$$
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = P^{-1}(X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$$
w' = P^{-1}(X^T X)^{-1} X^T y
$$
$$
w' = P^{-1}w
$$
$$




## Алгоритм преобразования

In [9]:
# разделим данные на фичи и таргеты

X_features = data.drop('Страховые выплаты', axis=1)
y_target = data['Страховые выплаты']

**Алгоритм**

In [10]:
# создадим класс LinearRegression_1 по формулам данным выше

class LinearRegression_1:
    def fit(self, features, target):
        X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
        y = target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, features):
        return features.dot(self.w) + self.w0
    
model = LinearRegression_1()
model.fit(X_features, y_target)
predictions = model.predict(X_features)
print(r2_score(y_target, predictions))

0.42494550286668


**Обоснование**

## Проверка алгоритма

In [11]:
# умножим признаки на обратимую матрицу

P = np.random.randint(100, size=(4, 4))
XP = X_features.dot(P)
display(XP)

,0,1,2,3
0,1885739.0,4118934.0,596043.0,1440900.0
1,1445018.0,3156344.0,456876.0,1104764.0
2,798638.0,1744450.0,252522.0,610740.0
3,1585074.0,3462238.0,500874.0,1210568.0
4,992447.0,2167740.0,313761.0,758616.0
...,...,...,...,...
4995,1357228.0,2964588.0,429000.0,1036988.0
4996,1991954.0,4350944.0,629460.0,1521644.0
4997,1288652.0,2814788.0,407256.0,984308.0
4998,1243133.0,2715372.0,392997.0,949668.0


In [12]:
# обучим модель и проыерим r2

model.fit(XP, y_target)
predictions = model.predict(XP)
print(r2_score(y_target, predictions))

0.4249455028664283


In [13]:
# проверим матрицу P на обратимость

P1 = np.linalg.inv(P)
print(np.linalg.det(P))
print(P1)
print()

E = P.dot(P1)
print(E)

4383492.000000002
[[ 0.03585087 -0.0283938   0.01931063 -0.03675289]
 [-0.01732591  0.00516985  0.00853566  0.01650191]
 [ 0.01158255 -0.00316643 -0.00928757  0.01058836]
 [-0.00218182  0.02371945 -0.01140734 -0.00345204]]

[[ 1.00000000e+00  0.00000000e+00 -5.55111512e-17 -2.77555756e-17]
 [ 2.77555756e-17  1.00000000e+00  1.11022302e-16  2.22044605e-16]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  5.55111512e-17]
 [-6.93889390e-17 -2.77555756e-17  3.46944695e-17  1.00000000e+00]]


## Вывод

1. Обучили модель с исходными признаки и получили коэффициент R2 = 0.42.
2. Согласно задаче обучения создали случайную обратимую матрицу, преобразовали исходные признаки, умножив их на матрицу.
3. Получили тот же коэффициент R2 = 0.42, обучив модель с преобразованными признаками